# MedGemma Clinical Note Assistant - Google Colab Setup

This notebook sets up and runs the FastAPI backend on Google Colab with GPU support.

## Prerequisites
- Google Colab Pro (recommended) for GPU access
- HuggingFace token (if model requires authentication)

## Step 1: Enable GPU Runtime

**Go to: Runtime → Change runtime type → GPU (T4 or better)**

Make sure to select GPU before running the cells below!

## Step 1: Install Dependencies

Install all required packages including FastAPI, PyTorch with CUDA, and transformers.

In [3]:
# Install dependencies
%pip install -q fastapi uvicorn[standard] pydantic pydantic-settings transformers torch accelerate pyngrok requests psutil

# Verify CUDA is available
import torch
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ CUDA Version: {torch.version.cuda}")
else:
    print("⚠️  CUDA not available. Make sure you selected GPU runtime!")

✅ CUDA available: False
⚠️  CUDA not available. Make sure you selected GPU runtime!


## Step 2: Clone from GitHub (Recommended)

**Option A: Clone from GitHub (Recommended)**
- If you've pushed your code to GitHub, clone it here
- This is the easiest and most reliable method

**Option B: Upload Files Manually**
- Use Colab's file uploader (right sidebar → 📁) to upload your `app/` directory

After cloning/uploading, your directory structure should include:
```
app/
  ├── main.py
  ├── api/
  ├── core/
  ├── services/
  ├── schemas/
  ├── templates/
  └── utils/
```

In [ ]:
# Option A: Clone from GitHub (Recommended)
# Replace with your actual GitHub repository URL and uncomment below:
# REPO_URL = "https://github.com/YOUR_USERNAME/YOUR_REPO_NAME.git"
# REPO_DIR = "YOUR_REPO_NAME"  # Directory name after cloning

# Example (uncomment and customize):
REPO_URL = "https://github.com/Aregawi-Teame/offline-clinical-note-assistant-backend.git"
REPO_DIR = "offline-clinical-note-assistant-backend"

import os
import subprocess
import shutil

# IMPORTANT: Reset to /content to ensure consistent path handling if cell is re-run
# This prevents nesting (cloning inside an existing clone)
if os.path.exists('/content'):
    os.chdir('/content')

# Define absolute path to ensure reliability
REPO_PATH = os.path.abspath(REPO_DIR)

# Check if repository directory already exists
if os.path.exists(REPO_PATH):
    print(f"📁 Repository '{REPO_DIR}' already exists.")

    # Check if it's a git repository
    git_dir = os.path.join(REPO_PATH, '.git')
    if os.path.exists(git_dir):
        print("   Detected git repository. Pulling latest changes...")

        # Change to repository directory first
        os.chdir(REPO_PATH)
        print(f"📂 Changed to: {os.getcwd()}")

        # Pull latest changes
        result = subprocess.run(['git', 'pull'], capture_output=True, text=True)
        if result.returncode == 0:
            print("✅ Successfully pulled latest changes")
            if result.stdout.strip():
                print(result.stdout)
            else:
                print("   (Already up to date)")
        else:
            print("⚠️  Could not pull changes")
            if result.stdout.strip():
                print(f"   Output: {result.stdout}")
            if result.stderr.strip():
                print(f"   Error: {result.stderr}")
    else:
        print(f"⚠️  '{REPO_DIR}' exists but is not a git repository")
        print(f"   Removing it and cloning fresh...")
        shutil.rmtree(REPO_PATH)
        # Clone fresh
        result = subprocess.run(['git', 'clone', REPO_URL], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Successfully cloned repository")
            os.chdir(REPO_PATH)
        else:
            print(f"❌ Failed to clone repository: {result.stderr}")

    # Ensure we're in the repo directory
    if os.getcwd() != REPO_PATH:
        os.chdir(REPO_PATH)
        print(f"📂 Current directory: {os.getcwd()}")
else:
    print(f"📥 Cloning repository from {REPO_URL}...")
    result = subprocess.run(['git', 'clone', REPO_URL], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ Successfully cloned repository")
        os.chdir(REPO_PATH)
        print(f"📂 Changed to directory: {os.getcwd()}")
    else:
        print(f"❌ Failed to clone repository")
        if result.stdout.strip():
            print(f"   Output: {result.stdout}")
        if result.stderr.strip():
            print(f"   Error: {result.stderr}")
        print("   Make sure the repository URL is correct and publicly accessible")

# Option B: If not using GitHub, upload files manually via Colab's file uploader
# Use the files tab (📁) in the left sidebar to upload your app/ directory

# Verify app directory exists
print("\n🔍 Verifying project structure...")
if os.path.exists('app'):
    print("✅ app/ directory found")
    app_contents = os.listdir('app')
    print(f"📁 Contents: {app_contents}")
    if 'main.py' in app_contents:
        print("✅ main.py found - project structure looks good!")
    else:
        print("⚠️  main.py not found - check your directory structure")
else:
    print("⚠️  app/ directory not found.")
    print("   Please either:")
    print("   1. Uncomment and customize REPO_URL above, then re-run this cell")
    print("   2. Upload files manually via Colab's file uploader")

## Step 3: Configure Environment

Set environment variables for the application. Colab will automatically use CUDA when `DEVICE=auto`.

### 🔑 HuggingFace Token (May be Required)

Some models require authentication. If you see errors about "not a valid model identifier", you may need a HuggingFace token:

1. **Get your token**: Go to https://huggingface.co/settings/tokens
2. **Create a token** (with "read" permissions)
3. **Set it in Cell 6** (see next cell) by uncommenting the `HUGGINGFACE_HUB_TOKEN` line

**Note**: Not all models require authentication. Try without a token first, and add it if you get authentication errors.

In [ ]:
import os
from google.colab import userdata
# Configuration - these will override .env file if present
# IMPORTANT: Set these BEFORE importing app modules (Settings reads env vars at import time)
os.environ['ENV'] = 'dev'  # Must be 'dev' or 'prod' - prevents validation errors
os.environ['DEVICE'] = 'auto'  # Will auto-detect CUDA in Colab
os.environ['MODEL_ID'] = 'google/medgemma-1.5-4b-it'  # or 'google/medgemma-1.5-4b-it'
os.environ['DEMO_MODE'] = 'false'  # Set to 'true' for demo mode (no model needed)
os.environ['MAX_NEW_TOKENS'] = '800'
os.environ['TEMPERATURE'] = '0.2'
os.environ['TOP_P'] = '0.9'
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

# IMPORTANT: HuggingFace Token (Required for some models)
# If you get "not a valid model identifier" errors, the model may require authentication
# 
# Get your token:
# 1. Go to: https://huggingface.co/settings/tokens
# 2. Create a token with "read" permissions
# 3. Uncomment the line below and paste your token
#
# os.environ['HUGGINGFACE_HUB_TOKEN'] = 'hf_your_token_here'
# 
# Alternative: Login using HuggingFace CLI
# !huggingface-cli login --token YOUR_TOKEN

# Check if token is set
hf_token = os.environ.get('HUGGINGFACE_HUB_TOKEN') or os.environ.get('HF_TOKEN')
if hf_token:
    print(f"✅ HuggingFace token is set (will be used for model authentication)")
else:
    print("ℹ️  HuggingFace token not set (may be required for some models)")

print("✅ Environment configured")
print(f"   ENV: {os.environ.get('ENV')} (must be 'dev' or 'prod')")
print(f"   DEVICE: {os.environ.get('DEVICE')}")
print(f"   MODEL_ID: {os.environ.get('MODEL_ID')} ⚠️  Make sure this matches what you want!")
print(f"   DEMO_MODE: {os.environ.get('DEMO_MODE')}")

# Verify the environment variable is set
model_id_set = os.environ.get('MODEL_ID', 'NOT SET')
print(f"\n📋 MODEL_ID is set to: {model_id_set}")
if model_id_set == 'NOT SET':
    print("⚠️  WARNING: MODEL_ID not set!")
elif 'medgemma-1.5-4b-it' in model_id_set or 'medgemma-4b-it' in model_id_set:
    print("   Try: 'google/medgemma-2b' or 'google/medgemma-7b'")
    print("   Search for valid models: https://huggingface.co/models?search=medgemma")
else:
    print(f"✅ MODEL_ID set to: {model_id_set}")

print("\n📌 IMPORTANT: Run this cell BEFORE starting the server (Cell 10)")
print("   The server must be restarted if you change MODEL_ID after it's running.")

## Step 4: Authenticate ngrok (Recommended)

ngrok requires authentication for reliable tunnels. You can use it without authentication, but authenticated sessions are more stable.

### Get ngrok Auth Token

1. **Sign up for free**: Go to https://dashboard.ngrok.com/signup
2. **Get your token**: After signing up, go to https://dashboard.ngrok.com/get-started/your-authtoken
3. **Copy the token** (looks like: `2abc123def456ghi789jkl_1a2B3c4D5e6F7g8H9i0J`)

### Authenticate in Colab

Run the cell below with your auth token. If you skip this, ngrok will still work but may have limitations.

In [ ]:
# ========================================
# INSTRUCTIONS: Paste Your ngrok Token Here
# ========================================
#
# After getting your token from https://dashboard.ngrok.com/get-started/your-authtoken:
# 1. Find the line below that says: # !ngrok config add-authtoken YOUR_NGROK_AUTH_TOKEN
# 2. Remove the # at the start (uncomment it)
# 3. Replace YOUR_NGROK_AUTH_TOKEN with your actual token (paste between quotes)
# 4. It should look like: !ngrok config add-authtoken 2abc123def456ghi789jkl_1a2B3c4D5e6F
# 5. Run this cell
#
# Example (after you paste your token):
# !ngrok config add-authtoken 2abc123def456ghi789jkl_1a2B3c4D5e6F7g8H9i0J
#
# ========================================

# 👇 PASTE YOUR TOKEN HERE 👇
# Uncomment the line below and replace YOUR_NGROK_AUTH_TOKEN with your actual token:
# !ngrok config add-authtoken YOUR_NGROK_AUTH_TOKEN

# Option: Skip authentication (works but may have limitations)
# If you skip authentication, you can still use ngrok but sessions may timeout sooner
# Just leave the line above commented and run this cell

print("ℹ️  ngrok authentication status:")
try:
    from pyngrok import ngrok
    # Try to check if authenticated (this is approximate)
    print("   Ready to authenticate")
    print("   ⚠️  To authenticate: Uncomment the line above, paste your token, and re-run this cell")
except:
    print("   Pyngrok not imported yet (will be imported in next cell)")

In [ ]:
# Diagnostic: Check tokenizer and model token configuration
# Run this if you get "CUDA device-side assert" errors

import os
os.environ.setdefault('ENV', 'dev')
os.environ.setdefault('MODEL_ID', 'google/medgemma-1.5-4b-it')
os.environ.setdefault('DEVICE', 'auto')

try:
    from transformers import AutoTokenizer, AutoModelForCausalLM
    import torch
    
    model_id = os.environ.get('MODEL_ID')
    hf_token = os.environ.get('HF_TOKEN')
    
    print(f"🔍 Diagnosing tokenizer/model: {model_id}")
    print()
    
    # Load tokenizer
    print("📥 Loading tokenizer...")
    if hf_token:
        tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    # Check token IDs
    print("🔢 Token Configuration:")
    print(f"   vocab_size: {tokenizer.vocab_size}")
    print(f"   pad_token_id: {tokenizer.pad_token_id}")
    print(f"   eos_token_id: {tokenizer.eos_token_id}")
    print(f"   bos_token_id: {tokenizer.bos_token_id}")
    print(f"   unk_token_id: {tokenizer.unk_token_id}")
    print()
    
    # Validate token IDs are within bounds
    print("✅ Validation:")
    issues = []
    
    if tokenizer.pad_token_id is not None:
        if tokenizer.pad_token_id >= tokenizer.vocab_size:
            issues.append(f"⚠️  pad_token_id ({tokenizer.pad_token_id}) >= vocab_size ({tokenizer.vocab_size})")
        else:
            print(f"   ✅ pad_token_id is valid (within vocab bounds)")
    else:
        print(f"   ℹ️  pad_token_id is None (will be set during model loading)")
    
    if tokenizer.eos_token_id is not None:
        if tokenizer.eos_token_id >= tokenizer.vocab_size:
            issues.append(f"⚠️  eos_token_id ({tokenizer.eos_token_id}) >= vocab_size ({tokenizer.vocab_size})")
        else:
            print(f"   ✅ eos_token_id is valid (within vocab bounds)")
    else:
        issues.append("⚠️  eos_token_id is None (this will cause issues)")
    
    if tokenizer.bos_token_id is not None:
        if tokenizer.bos_token_id >= tokenizer.vocab_size:
            issues.append(f"⚠️  bos_token_id ({tokenizer.bos_token_id}) >= vocab_size ({tokenizer.vocab_size})")
        else:
            print(f"   ✅ bos_token_id is valid")
    
    print()
    
    if issues:
        print("❌ Issues Found:")
        for issue in issues:
            print(f"   {issue}")
        print()
        print("💡 Solution: This tokenizer/model has out-of-bounds token IDs.")
        print("   Try a different model like 'google/gemma-2b-it' or 'google/gemma-7b-it'")
    else:
        print("✅ All token IDs are valid - no configuration issues detected")
        print("   If you still get CUDA errors, try:")
        print("   1. Set DEVICE=cpu in Cell 6")
        print("   2. Restart the runtime and try again")
    
except Exception as e:
    print(f"❌ Error during diagnostic: {e}")
    print("   Make sure you've run Cell 6 (Configure Environment) first")

In [ ]:
# Clean up any existing ngrok tunnels (optional - run if you get tunnel errors)
from pyngrok import ngrok
import time

print("🧹 Cleaning up existing ngrok tunnels...")

# Disconnect all visible tunnels
try:
    tunnels = ngrok.get_tunnels()
    for tunnel_info in tunnels:
        tunnel_url = tunnel_info.public_url if hasattr(tunnel_info, 'public_url') else str(tunnel_info)
        try:
            ngrok.disconnect(tunnel_url)
            print(f"   ✅ Closed: {tunnel_url}")
        except:
            pass
    if tunnels:
        time.sleep(2)
except:
    pass

# Kill any ngrok processes
try:
    import psutil
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            if 'ngrok' in proc.info['name'].lower():
                proc.kill()
                print(f"   ✅ Killed ngrok process (PID: {proc.pid})")
        except:
            pass
except ImportError:
    try:
        import subprocess
        subprocess.run(['pkill', '-9', '-f', 'ngrok'], capture_output=True, timeout=2)
    except:
        pass
except:
    pass

print("✅ Cleanup complete!")

## Step 5: Start FastAPI Server with ngrok

This will start the FastAPI server and create a public URL using ngrok. The server will run in the background.

In [ ]:
from pyngrok import ngrok
import uvicorn
import threading
import time
import requests
import os

# IMPORTANT: Verify MODEL_ID is set correctly BEFORE starting server
# The Settings class reads environment variables at import time (when server starts)
# So make sure Cell 6 (Configure Environment) was run FIRST
print("🔍 Verifying configuration before starting server...")
model_id_env = os.environ.get('MODEL_ID', 'NOT SET')
print(f"   Environment MODEL_ID: {model_id_env}")

if model_id_env == 'NOT SET':
    print("⚠️  WARNING: MODEL_ID not set! Run Cell 6 (Configure Environment) first!")
elif 'medgemma-1.5-4b-it' in model_id_env or 'medgemma-4b-it' in model_id_env:
    print(f"⚠️  WARNING: MODEL_ID '{model_id_env}' may not be valid!")
    print("   Try: 'google/medgemma-2b' or 'google/medgemma-7b'")
    print("   Check: https://huggingface.co/models?search=medgemma")
else:
    print(f"✅ MODEL_ID is set to: {model_id_env}")
    print(f"   (If you get model loading errors, verify this model exists on HuggingFace)")

print()

# Start ngrok tunnel
print("\n🚀 Starting ngrok tunnel...")
try:
    tunnel = ngrok.connect(8000)
except Exception as e:
    error_str = str(e)
    if "ERR_NGROK_334" in error_str or "already online" in error_str:
        print("\n❌ Error: A tunnel endpoint is already in use")
        print("   Solutions:")
        print("   1. Run the 'Kill ngrok tunnels' cell below, then re-run this cell")
        print("   2. Go to https://dashboard.ngrok.com/status/tunnels and close it manually")
        print("   3. Restart runtime (Runtime → Restart runtime)")
        raise
    elif "ERR_NGROK_324" in error_str or "limit" in error_str.lower():
        print("\n❌ Error: Too many tunnels (free tier limit: 5)")
        print("   Go to https://dashboard.ngrok.com/status/tunnels and close unnecessary tunnels")
        raise
    else:
        raise
# Extract the public URL string from the tunnel object
# The tunnel object string looks like: NgrokTunnel: "https://xxx.ngrok-free.dev" -> "http://localhost:8000"
if hasattr(tunnel, 'public_url'):
    public_url = tunnel.public_url
elif hasattr(tunnel, 'data') and 'public_url' in tunnel.data:
    public_url = tunnel.data['public_url']
else:
    # Extract URL from string representation
    tunnel_str = str(tunnel)
    import re
    url_match = re.search(r'"(https://[^"]+)"', tunnel_str)
    if url_match:
        public_url = url_match.group(1)
    else:
        public_url = tunnel_str  # Fallback

print(f"🌐 Public API URL: {public_url}")
print(f"📚 API Docs: {public_url}/api/v1/docs")
print(f"🔍 Health Check: {public_url}/api/v1/health")
print()
print("⏳ Starting server... (this may take 30-60 seconds on first run)")

# Start FastAPI server in background thread
def run_server():
    try:
        uvicorn.run(
            "app.main:app",
            host="0.0.0.0",
            port=8000,
            log_level="info"
        )
    except Exception as e:
        print(f"❌ Server error: {e}")

# Start server thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start and model to load
print("Waiting for server to initialize...")
time.sleep(10)  # Initial wait

# Try to check health
max_retries = 12
for i in range(max_retries):
    try:
        health_url = f"{public_url}/api/v1/health"
        response = requests.get(health_url, timeout=10)
        if response.status_code == 200:
            print("\n✅ Server is ready!")
            print(f"Response: {response.json()}")
            break
    except requests.exceptions.RequestException as e:
        if i < max_retries - 1:
            print(f"Waiting... ({i+1}/{max_retries}) - Server may still be loading model")
            time.sleep(5)
        else:
            print(f"\n⚠️  Health check timed out after {max_retries * 5} seconds")
            print(f"   Try accessing manually: {public_url}/api/v1/health")
            print(f"   Server is running - model may still be loading")

## Step 6: Test the API

Test the health endpoint and generate a clinical note.

In [ ]:
import requests
import json
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Health check
# Note: public_url should be set from the previous cell
# If you get an error, make sure you ran the "Start FastAPI Server" cell first
try:
    health_url = f"{public_url}/api/v1/health"
    
    # Try with SSL verification first, retry without if SSL error
    response = None
    for attempt in [True, False]:  # First with SSL, then without
        try:
            response = requests.get(health_url, timeout=10, verify=attempt)
            break
        except requests.exceptions.SSLError:
            if attempt:  # First attempt failed, try without SSL
                continue
            else:
                raise
        except Exception as e:
            raise
    
    if response:
        print("📊 Health Check:")
        print(json.dumps(response.json(), indent=2))
    else:
        print("❌ Failed to get health check response")
        
except NameError:
    print("❌ Error: public_url not found. Please run the 'Start FastAPI Server' cell first.")
except requests.exceptions.SSLError as e:
    print(f"❌ SSL Error: {e}")
    print("   💡 This is common with ngrok free tier. The API test cell handles this automatically.")
except requests.exceptions.ConnectionError as e:
    print(f"❌ Connection Error: {e}")
    print("   💡 Check if the server is running. Try restarting the 'Start FastAPI Server' cell.")
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"   Try accessing: {public_url}/api/v1/health manually")

In [ ]:
# Generate a SOAP note
url = f"{public_url}/api/v1/generate/"
payload = {
    "task": "SOAP",
    "notes": "Patient presents with chest pain. 45-year-old male with history of hypertension. Blood pressure 140/90, heart rate regular at 72 bpm.",
    "options": {
        "maxTokens": 800,
        "temperature": 0.2,
        "topP": 0.9
    }
}

print("🚀 Generating SOAP note...")
print(f"   Notes: {payload['notes'][:60]}...")
print(f"   ⏳ This may take 2-5 minutes on T4 GPU for {payload['options']['maxTokens']} tokens...")
print(f"   💡 Tip: Reduce 'maxTokens' to 400-600 for faster testing")
print()

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Try with SSL verification first, then without if SSL error occurs
max_retries = 3
for attempt in range(max_retries):
    try:
        # First attempt: with SSL verification
        verify_ssl = attempt == 0
        
        # Increased timeout for model generation (can take 3-5 minutes on T4 GPU)
        response = requests.post(
            url, 
            json=payload, 
            timeout=300,
            verify=verify_ssl
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Generation successful!")
            print(f"\n📝 Generated Note ({result['task']}):")
            print(f"\n{result['output']}")
            print(f"\n📊 Metadata:")
            print(f"   Model: {result['model']}")
            print(f"   Latency: {result['latencyMs']:.2f} ms")
            print(f"   Request ID: {result.get('requestId', 'N/A')}")
            break
        else:
            print(f"❌ Error {response.status_code}:")
            try:
                print(json.dumps(response.json(), indent=2))
            except:
                print(response.text)
            break
            
    except requests.exceptions.SSLError as e:
        if attempt == 0:
            print(f"⚠️  SSL error on attempt {attempt + 1}: {str(e)[:100]}...")
            print("   Retrying without SSL verification (ngrok free tier sometimes has SSL issues)...")
            continue
        else:
            print(f"❌ SSL error persisted after retries: {e}")
            print("\n💡 Troubleshooting SSL errors:")
            print("   1. Restart the ngrok tunnel (kill and restart the 'Start FastAPI Server' cell)")
            print("   2. Wait a few seconds and try again (ngrok may need time to stabilize)")
            print("   3. Check if the server is still running (try the health check cell)")
            print("   4. If using ngrok free tier, consider upgrading or authenticating ngrok")
            break
            
    except requests.exceptions.Timeout:
        print("⏱️  Request timed out after 5 minutes.")
        print("   💡 Try reducing 'maxTokens' to 400-600 for faster generation")
        print("   💡 Or wait longer - generation can take 5+ minutes on T4 GPU for 800 tokens")
        break
        
    except requests.exceptions.ConnectionError as e:
        if attempt < max_retries - 1:
            print(f"⚠️  Connection error on attempt {attempt + 1}: {str(e)[:100]}...")
            print(f"   Retrying in 2 seconds... ({attempt + 2}/{max_retries})")
            import time
            time.sleep(2)
            continue
        else:
            print(f"❌ Connection error after {max_retries} attempts: {e}")
            print("\n💡 Troubleshooting connection errors:")
            print("   1. Check if the server is running (try the health check cell)")
            print("   2. Verify the ngrok URL is correct (check the 'Start FastAPI Server' cell output)")
            print("   3. The ngrok tunnel may have disconnected - restart it")
            break
            
    except Exception as e:
        error_type = type(e).__name__
        if "SSL" in error_type or "ssl" in str(e).lower():
            if attempt == 0:
                print(f"⚠️  SSL-related error: {str(e)[:100]}...")
                print("   Retrying without SSL verification...")
                continue
        print(f"❌ Error: {e}")
        break

## Important Notes

### Session Management
- **Free Colab**: Sessions timeout after ~12 hours
- **Colab Pro**: Up to 24 hours (with idle timeout)
- Keep cells running or reconnect ngrok if session restarts

### GPU Access
- Colab Pro provides T4 GPU (sometimes A100)
- `DEVICE=auto` will automatically use CUDA when GPU is available
- First request is slower (model loading ~30-60 seconds)

### ngrok Authentication
- **Get free token**: Sign up at https://dashboard.ngrok.com/signup
- **Authenticate**: Use your token in Step 4 (cell above) before starting server
- **Why authenticate**: Longer tunnels, better stability, no random disconnects
- **Without auth**: Still works but may have shorter timeouts

### ngrok URL
- Free tier: URL changes on restart
- Paid tier: Can use fixed domain
- Save your public URL if you need to use it elsewhere

### Using the API from Outside Colab

Your API is now accessible via the public URL. Example curl command:
```bash
curl -X POST "YOUR_NGROK_URL/api/v1/generate/" \\
  -H "Content-Type: application/json" \\
  -d '{"task": "SOAP", "notes": "Your clinical notes here"}'
```